This tutorial can be found and ran in the GITHUB libray corrAI: https://github.com/BuildingEnergySimulationTools/corrai

---
# Characterisation method tutorial

The aim of this tutorial is to provide a complete workflow for material physical properties identification of a reference wall using **Modelitool** (modelica simulator) and **CorrAI** models by following these steps:

1. **Measurement import and verification**:
    - We will import measurement dataframe into the notebook and check measurement before selecting an analysis period.
    
2. **Model creation/import**:
    - We will either create a new mathematical model or impor model (FMU, openModelica).

3. **Performing sensitivity analysis**:
    - Sensitivity analysis will be conducted to determine how different input variables affect the output of the model. This step helps in identifying the most influential parameters in the model.

4. **Model identification**:
    - Using the results from the sensitivity analysis, we will identify the parameters that need to be adjusted to improve the model's accuracy. This involves fitting the model to the data and refining its parameters.

---

# Introduction

## Use case presentation

A "real-scale" test bench is used. The **O3BET** (or in this example BEF test bench in Anglet, France) offers experimental conditions to evaluate building façade solutions. Heat exchanges in a cell are restricted on five of its faces, while the sixth face is dedicated to the tested solution. Internal temperature and humidity conditions can be controlled or monitored. External conditions, including temperatures and solar radiation, are measured.

The tested technology here is a green façade, coupled wiht insulated panels. The experimental setup is presented in the following picture: one cell is equiped with the technology (right one) and another serves as a reference (insulation only).

| Figure : Pictures of reference wall and Urban Canopee installation |
| :---: |
| <img src="images/cladding.png"  style="height:200px;">  <img src="images/BEF_facades.jpeg"  style="height:200px;"> |

Sensors (heatflux density meters, thermocouples, RTD) are positioned in several parts: in the middle of insulation panels, between insulation and concrete layers, between leaves, in substrates, indoor. Climatic conditions (external temperature, incident solar radiation) are also monitored.


| Figure : Sensors installation scheme |
| :---: |
| <img src="images/sensors.png"  style="height:200px;">  <img src="images/Sensor_photo1.jpeg"  style="height:200px;"> <img src="images/Sensor_photo2.jpeg"  style="height:200px;">   | 

- Measure campaign spans from  april 2024 to october 2024
- Acquisition timestep is 60 secondes minimum.

## Identification framework
The following framework is proposed to identify the **REFERENCE** wall thermal conductivity, and provides:
- Physical model description using Python
- Sensitivity analysis to identify materials properties which have an influence on the discrepancy between model outputs and measured phenomenon
- Wall thermal conductivity identification using optimization algorithm

# 1. Measurement import and verification

First, let's load some generic libraries (pandas, Path). We can also define the function datetime_to_seconds, which will be later used to convert datetimes to seconds.


Then we can load the reference cell measurement data on python that will be used as boundary conditions. Note that the data loaded here should be cleaned beforhand (see Tutorial **"MeasuredDat example"** if needed)

In [ ]:
import pandas as pd
import os
import plotly.express as px
from pathlib import Path
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
import datetime as dt

def datetime_to_seconds(index_datetime):
    time_start = dt.datetime(index_datetime[0].year, 1, 1, tzinfo=dt.timezone.utc)
    new_index = index_datetime.to_frame().diff().squeeze()
    new_index.iloc[0] = dt.timedelta(
        seconds=index_datetime[0].timestamp() - time_start.timestamp()
    )
    sec_dt = [elmt.total_seconds() for elmt in new_index]
    return list(pd.Series(sec_dt).cumsum())

In [ ]:
TUTORIAL_DIR = Path(os.getcwd()).as_posix()

In [ ]:
reference_df = pd.read_csv(
    Path(TUTORIAL_DIR) / "resources/tuto_data_SA.csv",
    index_col=0,
    sep=";",
    decimal=".",
    parse_dates=True
)

In [ ]:
reference_df.head()

In [ ]:
reference_df.isna().any().any()

There seems to be no missing data. Let's first plot all data to check if nothing stands out. Although data was cleaned (supposedly, some measurement errors and irregularities might have been missed in the process.

In [ ]:
px.line(reference_df)

Here we can see that one of the temperature sensors installed in the insulation panels stopped measuring on september 7th. Only T_ins_2 will be used as the insulation temperature.

In [ ]:
reference_df.loc[:,"T_ins"] = reference_df['T_ins_2'] # middle of insulation temperature
reference_df.loc[:,"T_int"] =reference_df[['Tint_1', 'Tint_2']].mean(axis=1) # indoor temperature
reference_df.loc[:,"T_interface"] =  reference_df[['T_interface_1', 'T_interface_2']].mean(axis=1) # interface temperature, between insulation and concrete panels

For modeling purpose, we convert the temperature from °C to Kelvin.

In [ ]:
temperatures = [
    'T_ins',
    'T_int',
    'T_interface', 
    'T_ext'
]
reference_df[temperatures] += 273.15

Also, for the first simulation, let's chose a short period with both sunny and cloudy days:

In [ ]:
simulation_df = reference_df.loc["2024-09-04 00:00":"2024-09-09 00:00"]

# 2. Modeling approach and set-up

## 2.1 Proposed model 

For this example we propose a resistance/capacity approach. Based on electrical circuit analogy, each layer of the wall is modeled by two resistances and a capacity:


| Figure : RC model|
| :---: |
| <img src="images/RC_model.png"  style="height:400px;">   | 


Note that it is recommended to start with a **very simple** model (e.g., one resistance, one capacity with only Text and Tint as boundary conditions) and gradually make it more complex as you identify parameters. For the sake of this tutorial, the proposed model is already a bit detailed.


The following is a brief description of the thermal model:

- Each wall layer is modeled by 2 thermal resistances and a capacity.
    - Resistances to create a gradiant and better resolution of distribution of heat flow : $ R_1 = R_2 = \frac{ep_{layer}}{lambda_{layer} \times 2} $ 
    - Capacity in the middle of both our layers, representing its thermal mass and ability to store heat. : $ C = ep_{layer} \times rho_{layer} \times cap_{layer} $
 
- Inside and outside convection/conduction transfers are model as a constant value thermal resistance.

- Infrared transfers are considered :
    - With the sky, with $ T_{sky} = 0.0552T_{ext}^{1.5} $ as the sky is a significant source of infrared radiation, especially at night. This radiation can have a considerable impact on the thermal behavior of the system, influencing both heating and cooling processes
    - With the surrounding considered to be at $ T_{ext} $ as surroundings or environment also emit infrared radiation

- Short wave solar radiation heat flux is computed $Sw_{gain} = Pyr \times \alpha_{coat} $ with $Pyr$ the measured solar radiation onthe wall (W/m²) and  $\alpha_{coat}$ the coating solar absorbtion coefficient.

- Temperatures $ T_{ext}$ and $T_{int} $ are boundary conditions.  $ T_{int}$ represents the temperature within the controlled environment of the system.

Here are somes theoretical parameters for the model:

In [ ]:
# Surface of the tested wall
S_wall =  7

# Thickness of layer
ep_concrete = 0.200 #m
ep_ins = 0.140 #m

# Conductivity of layer
lambda_concrete = 0.13 # (W/mK)
lambda_ins = 0.031 # W/(mK)

# Density of layer
rho_concrete = 2400 # kg/m3 
rho_ins = 32.5  # kg/m3"


sc_concrete = 880 # J/kg.K
sc_ins = 1000 # J/kg.K

# solar paremetesr
alpha = 0.2 # absorption coefficient
epsilon = 0.8 # emissivity 
fview = 0.5 # view factor of tested wall

## 2.3 Define a simulator

You can either load a Modelica model or FMU (using **Modelitool** library, see: https://github.com/BuildingEnergySimulationTools/modelitool), 
or directly write a model on Python. For either, here are some in common parameters definition. 

For the model to work with corrAI and modelitool libraries (sensitivity analyses, optimisation, etc.), the model will be written as a class with :
- A parameter dictionary,
- Simulation options.

How does it work:

- Parameters: The wall's thermal resistances (R_ext, R_int, R_concrete, R_ins) and heat capacities (C_concrete, C_ins) are initialized with default values, but can be overridden by user inputs. Other physical parameters include the wall surface area (S_wall), view factor (fview), and material emissivity (epsilon).
- Dataframe: The model uses a dataframe input which includes external and internal temperatures (T_ext, T_int), time (time_sec), and solar radiation (Pyr from a pyranometer).
- Initial Conditions: The temperatures of the external surface, concrete, insulation, and interfaces are initialized based on the external and internal temperatures at the start.
- Time-Stepping: The simulation proceeds over time steps, updating temperatures at each layer.
- Radiative Heat Transfer: The model calculates radiative heat transfer between the wall surface and the sky, ambient, and direct solar radiation.
- Temperature Update: At each time step, temperatures are updated based on thermal resistances and capacitances of each layer using a finite difference approach.

In [ ]:
import numpy as np
import pandas as pd
from corrai.base.model import Model

class OpaqueWallSimple(Model):
    def simulate(
        self,
        property_dict: dict,
        simulation_options: dict,
        simulation_kwargs: dict | None = None,
        **kwargs,
    ) -> pd.DataFrame:

        default_parameters = {
            "R_ext": 0.005,
            "R_int": 0.01,
            "R_concrete": 0.10,
            "R_ins": 0.32,
            "C_concrete": 2.95e6,
            "C_ins": 3.64e4,
            "alpha": 0.2,
            "S_wall": 7,
            "epsilon": 0.4,
            "fview": 0.5,
        }
        parameters = {**default_parameters, **property_dict}

        R_ext       = parameters["R_ext"]
        R_int       = parameters["R_int"]
        R_concrete  = parameters["R_concrete"]
        R_ins       = parameters["R_ins"]
        C_concrete  = parameters["C_concrete"]
        C_ins       = parameters["C_ins"]
        alpha       = parameters["alpha"]
        S_wall      = parameters["S_wall"]
        epsilon     = parameters["epsilon"]
        fview       = parameters["fview"]

        sigma = 5.67e-8  # W/m^2/K^4

        df = simulation_options["dataframe"]
        time  = df["time_sec"].values
        T_ext = df["T_ext"].values
        T_int = df["T_int"].values
        Q_rad = df["Pyr"].values

        startTime = simulation_options.get("startTime", time[0])
        stopTime  = simulation_options.get("stopTime",  time[-1])

        mask  = (time >= startTime) & (time <= stopTime)
        time  = time[mask]
        T_ext = T_ext[mask]
        T_int = T_int[mask]
        Q_rad = Q_rad[mask]

        # init
        T_se        = np.zeros(len(time))
        T_concrete  = np.zeros(len(time))
        T_ins       = np.zeros(len(time))
        T_interface = np.zeros(len(time))
        T_si        = np.zeros(len(time))
        T_sky       = np.zeros(len(time))

        T_se[0]        = T_ext[0]
        T_concrete[0]  = 299
        T_ins[0]       = T_int[0]
        T_interface[0] = (T_ins[0] + T_concrete[0]) / 2
        T_si[0]        = T_int[0]
        T_sky[0]       = T_int[0]

        for t in range(1, len(time)):
            dt = time[t] - time[t - 1]

            T_sky[t] = 0.0552 * (T_ext[t] ** 1.5)

            Q_rad_sky = epsilon * fview * sigma * (T_se[t - 1] ** 4 - T_sky[t] ** 4) * S_wall
            Q_rad_amb = epsilon * fview * sigma * (T_se[t - 1] ** 4 - T_ext[t - 1] ** 4) * S_wall
            Q_rad_dir = Q_rad[t - 1] * alpha * S_wall

            T_se[t] = (
                T_ext[t - 1] / R_ext
                + T_ins[t - 1] / (R_ins / 2)
                + Q_rad_dir - Q_rad_sky - Q_rad_amb
            ) / (1 / R_ext + 1 / (R_ins / 2))

            T_interface[t] = (
                T_ins[t - 1] / (R_ins / 2) + T_concrete[t - 1] / (R_concrete / 2)
            ) / (1 / (R_concrete / 2) + 1 / (R_ins / 2))

            T_si[t] = (
                T_int[t - 1] / R_int + T_concrete[t - 1] / (R_concrete / 2)
            ) / (1 / R_int + 1 / (R_concrete / 2))

            T_ins[t] = T_ins[t - 1] + dt / C_ins * (
                (T_se[t] - T_ins[t - 1]) / (R_ins / 2)
                + (T_interface[t] - T_ins[t - 1]) / (R_ins / 2)
            )

            T_concrete[t] = T_concrete[t - 1] + dt / C_concrete * (
                (T_interface[t] - T_concrete[t - 1]) / (R_concrete / 2)
                + (T_si[t] - T_concrete[t - 1]) / (R_concrete / 2)
            )

        # output
        df_out = pd.DataFrame(
            {
                "T_concrete":  T_concrete,
                "T_interface": T_interface,
                "T_ins":       T_ins,
            },
            index=df.index[mask],
        )
        self.simulation_options = simulation_options
        return df_out

Datetime should be in second: we can use datetime_to_seconds function defined earlier. Moreover, data are in minutes, we should resample them to 5min samples.

In [ ]:
simulation_df.loc[:,"time_sec"] = datetime_to_seconds(simulation_df.index)
simulation_df_resample = simulation_df.resample("5min").mean()

Now, let's define: 
- simulation options, with starttime, endtime, and a dataframe for boundary conditions
- a dictionary, containing values for ou different parameters

In [ ]:
second_index = datetime_to_seconds(simulation_df_resample.index)

In [ ]:
simulation_options_PYTH={
    "dataframe":simulation_df_resample,
    "startTime": second_index[0],
    "endTime": second_index[-1],  
}

In [ ]:
parameter_dict_PYTH = {
    "R_ext": 0.04/S_wall,       
    "R_int": 0.13/S_wall,      
    "R_concrete": 1 / (lambda_concrete / ep_concrete) / 2 / S_wall,   
    "R_ins": 1 / (lambda_ins / ep_ins) / 2 / S_wall, 
    "C_ins": rho_ins*ep_ins*S_wall*sc_ins,  
    "C_concrete": rho_concrete*ep_concrete*S_wall*sc_concrete,       
    "alpha": alpha,       
    "S_wall": S_wall,         
    "epsilon": epsilon,
    'fview': fview
}

We can now instantiate the model and run the simulation. 

In [ ]:
simu_PYTH = OpaqueWallSimple()

init_res_PYTH = simu_PYTH.simulate(
    property_dict=parameter_dict_PYTH,
    simulation_options=simulation_options_PYTH
)

In [ ]:
init_res_PYTH.head()

Let's compare results to measurement:

In [ ]:
#renaming
copy_res = init_res_PYTH
copy_res.index = copy_res.index.tz_localize(None)

copy_res = copy_res.rename(columns={
    "T_concrete": "T_concrete_PYTHON",
    "T_interface": "T_interface_PYTHON",
    "T_ins": "T_insulation_PYTHON",
})

In [ ]:
measure_comp = pd.concat([
    simulation_df_resample[["T_interface", "T_ins"]], 
    copy_res[["T_interface_PYTHON", "T_insulation_PYTHON" ]]], axis = 1)

color_map = {
    "T_interface": "darkblue", 
    "T_interface_PYTHON": "blue", 
    "T_ins": "darkgreen", 
    "T_insulation_PYTHON": "green" 
}

fig = px.line(measure_comp)

for trace in fig.data:
    trace_name = trace.name
    if trace_name in color_map:
        trace.line.color = color_map[trace_name]
        
fig.show()

Not  good. A sensitivity analysis should be performed to rank the parameters by order of influence on the error between measured temperature and model prediction.


# 3. Sensitivity analysis
It is very important to know how our defined parameters have an influence on the model prediction. Therefore, we use a sensitivity analysis to "rank" the parameter by order of influence
on the model error. 

## 3.1. Error function
The chosen error function is the CV_RMSE. The formula for CV_RMSE is given by:

$$
CV\_RMSE = \frac{RMSE}{\bar{y}}
$$

Where:
- *RMSE* is the root mean squared error,
- *bar{y}* is the mean of the observed values.

The RMSE is calculated as:

$$
RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$

Where:
- *n* is the number of observations,
- *y_i* is the observed value for the \( i \)-th observation,
- *hat{y}_i* is the predicted value for the \( i \)-th observation.

The CV_RMSE measures the variation of the RMSE relative to the mean of the observed values. It provides a standardized measure of the error, which can be useful for comparing the performance of different models across different datasets.
Here, we can chose the error function as the CV_RMSE between measured temperature(s) and model prediction.

## 3.2. Tested parameters

The chosen parameters are all the model parameters θ=(R_concrete, R_ins, C_ins, C_concrete, alpha, epsilon, R_ext, R_int ).
Each decision variable of the optimization problem must be described using a `Parameter` object.
A parameter specifies:
* `name` — The variable name (string, must be unique within the problem).
* `ptype` — Variable type, one of:
    * `"Real" `— Continuous real number
    * `"Integer"` — Discrete integer
    * `"Binary"` — Boolean, domain {False, True} (set automatically if no domain is given)
    *` "Choice"` — Categorical variable with a fixed set of discrete options
* `Domain definition` — Choose exactly one of:
    * ` interval=(lo, hi) `— Lower and upper bounds (for "Real" and "Integer", optional for "Binary" if you want (0,1))
    * ` values=(v1, v2, …)` — Explicit list/tuple of allowed values (for "Choice", and optionally for "Integer", "Real", or "Binary")
*` Optional fields`:
    * `init_value` — Initial value (or tuple/list of initial values for batch runs); must be within the defined domain
    * `relabs` — `"Absolute"` or `"Relative"` (or a boolean flag, depending on usage in your model)
    * `model_property` — String or tuple specifying the corresponding property in the simulation/model
    * `min_max_interval` — Optional extra bounds for use in analysis/validation

In [ ]:
from corrai.base.parameter import Parameter

In [ ]:
frac_p = 0.20 # 20%
frac_conv = 0.40 # 40%

params = [
    Parameter(
        name='R_concrete',
        interval=(1-frac_p, 1+frac_p),
        ptype="Real",
        init_value=1 / (lambda_concrete / ep_concrete) / 2 / S_wall,
        relabs="Relative",
        model_property='R_concrete',
    ),
    Parameter(
        name='R_ins',
        interval=(1-frac_p, 1+frac_p),
        init_value=1 / (lambda_ins / ep_ins) / 2 / S_wall,
        relabs="Relative",
        ptype="Real",
        model_property='R_ins',
    ),
    Parameter(
        name='C_ins',
        interval=(1-frac_p, 1+frac_p),
        init_value=rho_ins*ep_ins*S_wall*sc_ins,
        relabs="Relative",
        ptype="Real",
        model_property='C_ins',
    ),
    Parameter(
        name='C_concrete',
        interval=(1-frac_p, 1+frac_p),
        init_value=rho_concrete*ep_concrete*S_wall*sc_concrete,
        relabs="Relative",
        ptype="Real",
        model_property='C_concrete',
    ),
    Parameter(
        name='alpha',
        interval=(0.1, 0.6),
        ptype="Real",
        model_property='alpha',
    ),
    Parameter(
        name='epsilon',
        interval=(0.2, 0.9),
        ptype="Real",
        model_property='epsilon',
    ),
    Parameter(
        name='R_ext',
        init_value= 0.04/S_wall,
        interval=(1-frac_conv, 1+frac_conv),
        ptype="Real",
        relabs="Relative",
        model_property='R_ext',
    ),
    Parameter(
        name='R_int',
        init_value= 0.13/S_wall,
        interval=(1-frac_conv, 1+frac_conv),
        ptype="Real",
        relabs="Relative",
        model_property='R_int',
    ),
]

## 3.3. Problem description
We can now load from `corrai.sensitivity` module a sensitivity method.

*Note: for now, only <code>SOBOL</code>, <code>FAST</code>, <code>RBD_FAST</code>, 
and <code>MORRIS</code> methods are implemented.*

### 3.1.A SOBOL method

In [ ]:
from corrai.sensitivity import SobolSanalysis

In [ ]:
sa_study = SobolSanalysis(
    parameters=params,
    model=simu_PYTH,
    simulation_options=simulation_options_PYTH,
)

The method add sample draw a sample of parameters to be simulated. Each method has its sampling method. Please see SALib documentation for further explanation (https://salib.readthedocs.io/en/latest/index.html)

Note that:
- Convergence properties of the Sobol' sequence is only valid if
        `N` (100) is equal to `2^n`.
        N (int) – The number of samples to generate. Ideally a power of 2 and <= skip_values.
- Convergence properties of the Fast' method is only valid if sample size N > 4M^2 (M=4 by default)

In [ ]:
sa_study.add_sample(N=2**6,  n_cpu=-1)

First, we can first take a look on the parallel coordinate plot of all parameter values and one of the simulation outputs aggregated according to a chosen aggregation method:

In [ ]:
sa_study.plot_pcp(
    aggregations={"T_ins": np.mean},
)

Now, let's perform a **Sobol sensitivity analysis** on the following indicator: cv_rmse on `T_ins` .
The `analyze()` method computes Sobol indices based on a chosen performance metric—in this case, the **cv_rmse**—by comparing model predictions of `T_ins` against the provided reference time series (measured or baseline data).

In [ ]:
sa_study.analyze(
    indicator="T_ins",
    method="cv_rmse",
    reference_time_series=simulation_df_resample["T_ins"],
)

The `plot_bar()` method displays the Sobol sensitivity indices as a bar chart, allowing a quick comparison of parameter importance.
It uses the aggregated results from the `analyze()` step (e.g., based on `cv_rmse`) and shows  first-order indices for each parameter, making it easy to identify which parameters most influence the chosen performance metric.

The sum of all the indices should be close to 1. Also, the mean confidence interval should be very low. In that case, results of the sensitivity analysis can be considered as robust.

In [ ]:
sa_study.plot_bar("T_ins")

The parameter epsilon appears to have the most influence on the model errors. This impact is calculated on the overall error, but will depend on the time of the day. Let's observe the parameters' impact dynamically with a 15minutes frequency.

The `plot_dynamic_metric()` method extends the analysis to a frequency view of the selected performance metric.
Here, we apply it to `T_ins` using the `cv_rmse` metric, comparing model output against the `reference_time_series` at regular intervals.

Key arguments:
- **`freq`**: Controls the temporal resolution for aggregation (e.g., `freq="2h"` computes sensitivity indices every two hours). This enables tracking how parameter influence changes over time.
- **`method`**: The metric used for comparison (here, `cv_rmse` between model predictions and the reference).
- **`reference_time_series`**: The baseline data against which each simulation is evaluated.

This approach is particularly useful in dynamic systems, where parameter importance may vary significantly across different time periods.

*Calculation can take time according to the number of simulations and frequency of data.*


In [ ]:
sa_study.plot_dynamic_metric("T_ins", freq="2h", method="cv_rmse", reference_time_series=simulation_df_resample["T_ins"] )

### 3.1.B MORRIS method

Note that a first screening using <code>MORRIS</code> method (as well as `RBD-FAST` and `RBD` methods)  could have been performed.

For <code>MORRIS</code> method, two indices, µj* for the mean of the absolute values of these effects and σj for the standard deviation of these effects, are calculated as follows:

$$
mu_{j}^{*} = \frac{1}{r} \sum_{i=1}^{r} E_{ij}
$$

$$
sigma_{j} = \sqrt{\frac{1}{r-1} \sum_{i=1}^{r} (E_{ij} - \mu_{j}^{*})^2}
$$

In [ ]:
from corrai.sensitivity import MorrisSanalysis

sa_study = MorrisSanalysis(
    parameters=params,
    model=simu_PYTH,
    simulation_options=simulation_options_PYTH,
)

In [ ]:
sa_study.add_sample(N=3**2, n_cpu=-1)

We can plot all simulations in one graph and compare the simulated internal temperature to measured T_int. Argument <code>show_legends</code> can be set to True if you want see associated parameters values.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from corrai.sampling import plot_sample

plot_sample(
    results=sa_study.results,
    indicator="T_ins",
    reference_timeseries = simulation_df_resample["T_ins"],
    x_label="time",
    y_label="simulated temperature of T_ins [K]",
    show_legends=False
)

This graph can be very instructive as at some moments, simulations are far from measurements. It show that whatever the values of our parameters, it still does not fit reality: this is either due to a problem of measurement, or to our modeling approach (physical inconsistency, physical phenomenon not properly taken into account, etc.).


In [ ]:
sa_study.evaluate(
    model = simu_PYTH,
    simulation_options=simulation_options_PYTH,
)

The highter $mu_{j}$, the more the parameter $j$ contributes to an uncertain output, and the higher $sigma_{j}$, the more pronounced the interaction effects between the model parameters are. Plotting $sigma_{j}$ against $mu_{j}^{}$ is often used to distinguish factors with negligible, linear, and/or interaction effects.

In [ ]:
sa_study.plot_scatter(indicator="T_ins")

Besides this visual interpretation, it is also possible to calculate the Euclidean distance $d$ to the origin to obtain the total effect of the uncertain parameters:

In [ ]:
sa_study.plot_bar(indicator="T_ins")

Five parameters seem to have more impact on the error than other: $alpha$, $R_{ins}$, $epsilon$, $R_{ext}$, and $R_{concrete}$. 
These results are consistant with <code>SOBOL</code> method.

## 3.4 Conclusion on sensitivity analysis

The sensitivity analysis allows us to rank the influence of uncertain parameter
on an indicator. 

Results with Sobol are consistant with Morris.  $epsilon$ and $alpha$ are the most influencial parameters, followed by $R_{ins}$, $R_{ext}$, and $R_{concrete}$.

In the following section, we will see how to use corrai to identify the
optimal values for these parameters in order to fit the measurement.

# 4. Identification
Now, we proceed to finding optimal values for these parameters by minimizing the coefficient of variation of root mean square error (cv_rmse) between one or several measured nodes, and one or several relevant outputs of our model.


### Step-by-Step Process

1. **Define the model and parameters**: by defining `OpaqueWallSimple`(this we already did) and specifying the parameters to be identify. Each parameter includes a name, an interval of possible values, a type, and an initial value.

2. **Instantiate an objective function**: We create an instance of the `ObjectiveFunction` class, providing the model, simulation options, list of parameters, and indicators. The `scipy_obj_function` method of `ObjectiveFunction` will be used as the objective function for optimization. This method calculates the cv_rmse for the given parameter values.

3. **Perform optimization**: We use the `minimize_scalar` function from `scipy.optimize` to minimize the objective function. Different methods can be chosen for optimization, such as Brent, bounded, and golden.


## 4.1. Objective function
In parameter optimization, we aim to adjust certain model parameters to minimize the difference between simulated and observed data. The objective function is a scalar function that quantifies this difference. In this case, the CV_RMSE (Coefficient of Variation of Root Mean Square Error) is used as a measure of how well the model output matches the reference measurements.

**How the ObjectiveFunction Class Works** : The ObjectiveFunction class simplifies the process of optimizing model parameters by providing a structured way to:

- Run simulations: For a given set of parameters, the model is simulated over the input data.
- Calculate error metrics (e.g., CV_RMSE): The model output is compared to the reference measurements, and a scalar error metric is calculated (such as the CV_RMSE).
Optimize: The class can then be used with optimization algorithms (like scipy.optimize or pymoo) to adjust the model parameters in order to minimize the error metric.
Attributes of the ObjectiveFunction Class

To do so, the  `ObjectiveFunction` class is designed to facilitate the optimization of model parameters using `scipy.optimize` or `pymoo` optimization methods by encapsulating the logic for simulation and indicator calculation. The `ObjectiveFunction` class takes a model, simulation options, a list of parameters to be calibrated, and a list of indicators as input. It provides methods to calculate the objective function, which can be used by optimization routines to find the optimal parameters.

### Attributes

- **model**: The model to be calibrated.
- **simulation_options**: A dictionary containing simulation options, including input data.
- **param_list**: A list of dictionaries specifying the parameters to be calibrated.
- **indicators_config**: A dictionary where the keys are the names of the indicators corresponding to the model outputs, and the values are either:
    - An aggregation function to compute the indicator (e.g., np.mean, np.sum).
    - A tuple (function, reference_data) if a comparison with reference data is required (e.g., sklearn.metrics.mean_squared_error, corrai.metrics.mae).
- **scipy_obj_indicator**: The name of the indicator used as the objective function for optimization with scipy.optimize. By default, it is the first key in indicators_config.

Let's define an anlysis daterange and redefine simulation optiouns:

In [ ]:
feat_train = simulation_df_resample.loc["2024-09-04 00:00":"2024-09-07 00:00"]

second_index = datetime_to_seconds(feat_train.index)

simulation_options_PYTH = {
    "dataframe": feat_train,
    "startTime": second_index[0],
    "endTime": second_index[-1],
}

## 4.2. One-dimensional optimization problem

First, we can try scalar functions optimization from scipy (different methods: brent, boulded, golden ... see documentation on scipy website).
For Scypi, each objective function is minimized for optimization:
- Here we chose as indicators the temperature calculated and measured within the wall insulation. Note this could be another node (a heat flux densitiy, another temperature node).
- The identified parameter is  $alpha$.

Let's define a reference dictionnary, setting observation and prediction to be used for the CV_RMSE calculation. (Here they have have the same name, so it can be confusing)

In [ ]:
from corrai.base.metrics import cv_rmse

reference_dict = {"T_ins": (cv_rmse, feat_train["T_ins"])}

We also need to set which parameter should be calibrated.

In [ ]:
calibration_params = [Parameter(
    name='epsilon',
    interval=(0.1, 0.8),
    ptype="Real",
    model_property='epsilon'
)]

We can now instanciate an objective function, using `ObjectiveFunction`.

In [ ]:

from corrai.surrogate import ObjectiveFunction

obj_func = ObjectiveFunction(
    model=OpaqueWallSimple(),
    simulation_options=simulation_options_PYTH,
    parameters=calibration_params,
    indicators_config=reference_dict,
)

The `minimize_scalar` function in `scipy.optimize` is used for scalar function minimization, specifically for one-dimensional optimization problems. This function finds the minimum value of a scalar function over a specified interval. The main methods are `Brent`, `bounded`, and `golden`.

The function returns an optimization result object that contains information about the optimization process and the final solution.

- **Brent** :The Brent method uses Brent’s algorithm, which combines a parabolic interpolation with the golden section search. This method does not require the interval bounds.
- **Golden** : Employs the golden section search method, which reduces the interval of uncertainty using the golden ratio. Simple and reliable for unimodal functions, but may be slower than Brent's method.
- **Bounded** : Restricts the search to the specified bounds using a combination of golden section search and parabolic interpolation.
Advantages: Ensures that the solution remains within the given bounds, making it ideal for constrained problems.

In [ ]:
from scipy.optimize import minimize_scalar

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

result = minimize_scalar(
    obj_func.scipy_obj_function, 
    bounds=obj_func.bounds[0],
    method="Bounded"
)

result

A solution is found with a value of 0.12 for alpha and 0.36 for the CV_RMSE. Let's check if the parameter value is close to the boundaries.

In [ ]:
obj_func.bounds

It is indeed not far from 0.1 but not a the limit. Let's now run the simulation using this parameter value and compare with the initial simulation.

In [ ]:
parameter_names = [param.name for param in calibration_params]
parameter_dict1 = {param_name: result.x for i, param_name in enumerate(parameter_names)}

result_optim = simu_PYTH.simulate(
    property_dict=parameter_dict1,
    simulation_options=simulation_options_PYTH
)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=init_res_PYTH.index,
    y=simulation_df_resample["T_ins"],
    fill=None,
    mode='lines',
    line_color='green',
    name="T_insulation - Measurement"
))

fig.add_trace(go.Scatter(
    x=init_res_PYTH.index,
    y=init_res_PYTH["T_ins"],
    fill=None,
    mode='lines',
    line_color='orange',
    name="T_insulation - Initial results"
))

fig.add_trace(go.Scatter(
    x=init_res_PYTH.index,
    y=result_optim["T_ins"],
    fill=None,
    mode='lines',
    line_color='brown',
    name="T_insulation - Optimization results"
))


fig.update_layout(
    title='Optimization vs. Measurement ',
    xaxis_title='Date',
    yaxis_title='Temperature [K]')

fig.show()

Results are closer to measurements but still far off.

## 4.3. Multi- objectives and parameters optimization

Let's use Pymoo, integrated into the `Problem` class of `CorrAI`  and multi-parameters optimization with multi_objectives.

Note that :
- All **objectives are minimized**.
- All **constraints** must be provided in the form **g(x) ≤ 0** (inequalities).

`Problem` aggregates all evaluator outputs into a single dictionary and then **extracts**:
- **Objectives** in the order of `objective_ids` → matrix **F**
- **Constraints** in the order of `constraint_ids` → matrix **G** (≤ 0)

`Problem` takes as arguments:
- **`parameters`**: as defined earlier from the class `Parameter`
- **`evaluators`**  (or objective functions)
- **`objective_ids`** :   Names (keys) to extract from evaluator outputs to build **F** (in order).
- **`constraint_ids`**  : Optional names to extract for **G** (inequalities ≤ 0). If omitted, `G` is empty.


### 4.3.1. Let's try with few parameters

In [ ]:
frac_p = 0.20

calibration_params = [

    Parameter(
        name='R_ins',
        interval=(1-frac_p, 1+frac_p),
        init_value=1 / (lambda_ins / ep_ins) / 2 / S_wall,
        relabs="Relative",
        ptype="Real",
        model_property='R_ins',
    ),
    Parameter(
        name='alpha',
        interval=(0.1, 0.9),
        ptype="Real",
        model_property='alpha',
    ),
    Parameter(
        name='epsilon',
        interval=(0.1, 0.9),
        ptype="Real",
        model_property='epsilon',
    ),
]

Here, we can add the interferace temperature (between insulation and concrete panels) as an indicator.

In [ ]:
from corrai.base.metrics import cv_rmse

obj_func = ObjectiveFunction(
    model=OpaqueWallSimple(),
    simulation_options=simulation_options_PYTH,
    parameters=calibration_params,
    indicators_config={
        "T_ins": (cv_rmse, feat_train["T_ins"]),
        "T_interface": (cv_rmse, feat_train["T_interface"]),
    },
    scipy_obj_indicator="T_ins",
)

Now let's instanciate the problem using the classe `Problem`:

In [ ]:
from corrai.multi_optimize import Problem

problem = Problem(
    parameters=calibration_params,
    evaluators=[obj_func],
    objective_ids=["T_ins", "T_interface"],
)

For a two objective problem, we choose here **NSGA2**, as a well-known multi-objective optimization algorithm based on non-dominated sorting and crowding.
List of algorithms here https://pymoo.org/algorithms/list.html#nb-algorithms-list.

If the verbose=True, some printouts during the algorithm’s execution are provided. This can very from algorithm to algorithm. Here, we execute NSGA2 on a problem where pymoo has no knowledge about the optimum. Each line represents one iteration. The first two columns are the current generation counter and the number of evaluations so far. For constrained problems, the next two columns show the minimum constraint violation (cv (min)) and the average constraint violation (cv (avg)) in the current population. This is followed by the number of non-dominated solutions (n_nds) and two more metrics which represents the movement in the objective space.

In [ ]:

from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM

In [ ]:
algorithm = NSGA2(
    pop_size=50,
    #n_offsprings=10,
    #sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True
)

Here, we will run 15 generation with a population of 50.

In [ ]:
termination = get_termination("n_gen", 15)

res = minimize(problem,
               algorithm,
               termination,
               seed=42,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Let's visualize the objectives functions results.

In [ ]:
from pymoo.visualization.scatter import Scatter
Scatter().add(res.F).show()

For a bi-objective problem,and helping us chosing the best set of parameters value, we can use the decomposition method called Augmented Scalarization Function (ASF), a well-known metric in the multi-objective optimization literature.
Let us assume the are equally important by setting the weights to 0.5 and 0.5 and setting these

In [ ]:
from pymoo.decomposition.asf import ASF
F = res.F
approx_ideal = F.min(axis=0)
approx_nadir = F.max(axis=0)
nF = (F - approx_ideal) / (approx_nadir - approx_ideal)

fl = nF.min(axis=0)
fu = nF.max(axis=0)
weights = np.array([0.5, 0.5])
decomp = ASF()

i = decomp.do(nF, 1/weights).argmin()

parameter_names = [param.name for param in calibration_params]
parameter_dict = {param_name: res.X[i][j] for j, param_name in enumerate(parameter_names)}

print(
    "Best regarding ASF: Point \ni = %s\nF = %s" % (i,  F[i]),
    parameter_dict
)


The estimated parameters seem consistent with our expectations. We can compare the profile of measured indoor temperature with the output that the model predicts given the identified optimal parameters. 

In [ ]:
result_optim = simu_PYTH.simulate(
    property_dict=parameter_dict,
    simulation_options=simulation_options_PYTH,
)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=init_res_PYTH.index,
    y=simulation_df_resample["T_ins"],
    fill=None,
    mode='lines',
    line_color='green',
    name="T_insulation - Measurement"
))

fig.add_trace(go.Scatter(
    x=init_res_PYTH.index,
    y=init_res_PYTH["T_ins"],
    fill=None,
    mode='lines',
    line_color='orange',
    name="T_insulation - Initial results"
))


fig.add_trace(go.Scatter(
    x=init_res_PYTH.index,
    y=result_optim["T_ins"],
    fill=None,
    mode='lines',
    line_color='brown',
    name="T_insulation - Optimization results"
))


fig.update_layout(
    title='Optimization vs. Measurement ',
    xaxis_title='Date',
    yaxis_title='Temperature [K]')

fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()


fig.add_trace(go.Scatter(
    x=init_res_PYTH.index,
    y=simulation_df_resample["T_interface"],
    fill=None,
    mode='lines',
    line_color='green',
    name="T_interface - Measurement"
))

fig.add_trace(go.Scatter(
    x=init_res_PYTH.index,
    y=init_res_PYTH["T_interface"],
    fill=None,
    mode='lines',
    line_color='orange',
    name="T_interface - Initial results"
))

fig.add_trace(go.Scatter(
    x=init_res_PYTH.index,
    y=result_optim["T_interface"],
    fill=None,
    mode='lines',
    line_color='brown',
    name="T_interface - Optimization results"
))

fig.update_layout(
    title='Optimization vs. Measurement ',
    xaxis_title='Date',
    yaxis_title='Temperature [K]')

fig.show()

### 4.3.2. All relevant parameters

We can try to identify all five parameters at once, with larger intervals.

In [ ]:
frac_p = 0.50
frac_conv = 0.50

calibration_params = [
    Parameter(
        name='R_concrete',
        interval=(1-frac_p, 1+frac_p),
        ptype="Real",
        init_value=1 / (lambda_concrete / ep_concrete) / 2 / S_wall,
        relabs="Relative",
        model_property='R_concrete',
    ),
    Parameter(
        name='R_ins',
        interval=(1-frac_p, 1+frac_p),
        init_value=1 / (lambda_ins / ep_ins) / 2 / S_wall,
        relabs="Relative",
        ptype="Real",
        model_property='R_ins',
    ),
    Parameter(
        name='alpha',
        interval=(0.1, 0.9),
        ptype="Real",
        model_property='alpha',
    ),
    Parameter(
        name='epsilon',
        interval=(0.1, 0.9),
        ptype="Real",
        model_property='epsilon',
    ),
    Parameter(
        name='R_ext',
        init_value= 0.04/S_wall,
        interval=(0.02/S_wall, 0.06/S_wall),
        ptype="Real",
        model_property='R_ext',
    ),
]

obj_func = ObjectiveFunction(
    model=OpaqueWallSimple(),
    simulation_options=simulation_options_PYTH,
    parameters=calibration_params,
    indicators_config={
        "T_ins": (cv_rmse, feat_train["T_ins"]),
        "T_interface": (cv_rmse, feat_train["T_interface"]),
    },
    scipy_obj_indicator="T_ins",
)

problem = Problem(
    parameters=calibration_params,
    evaluators=[obj_func],
    objective_ids=["T_ins", "T_interface"],
)

algorithm = NSGA2(
    pop_size=100,
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True
)

termination = get_termination("n_gen", 10)

res = minimize(problem,
               algorithm,
               termination,
               seed=42,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

In [ ]:
from pymoo.visualization.scatter import Scatter
Scatter().add(res.F).show()

In [ ]:
from pymoo.decomposition.asf import ASF
F = res.F
approx_ideal = F.min(axis=0)
approx_nadir = F.max(axis=0)
nF = (F - approx_ideal) / (approx_nadir - approx_ideal)

fl = nF.min(axis=0)
fu = nF.max(axis=0)
weights = np.array([0.9, 0.1])
decomp = ASF()

i = decomp.do(nF, 1/weights).argmin()

parameter_names = [param.name for param in calibration_params]
parameter_dict2 = {param_name: res.X[i][j] for j, param_name in enumerate(parameter_names)}

print("Best regarding ASF: Point \ni = %s\nF = %s" % (i, F[i]),
      parameter_dict2
     )

In [ ]:
result_optim2 = simu_PYTH.simulate(
    property_dict=parameter_dict2,
    simulation_options=simulation_options_PYTH,
)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=init_res_PYTH.index,
    y=simulation_df_resample["T_ins"],
    fill=None,
    mode='lines',
    line_color='green',
    name="T_insulation - Measurement"
))

fig.add_trace(go.Scatter(
    x=init_res_PYTH.index,
    y=init_res_PYTH["T_ins"],
    fill=None,
    mode='lines',
    line_color='orange',
    name="T_insulation - Initial results"
))


fig.add_trace(go.Scatter(
    x=init_res_PYTH.index,
    y=result_optim["T_ins"],
    fill=None,
    mode='lines',
    line_color='brown',
    name="T_insulation - Optimization results 1"
))

fig.add_trace(go.Scatter(
    x=init_res_PYTH.index,
    y=result_optim2["T_ins"],
    fill=None,
    mode='lines',
    line_color='pink',
    name="T_insulation - Optimization results 2"
))

fig.update_layout(
    title='Optimization vs. Measurement ',
    xaxis_title='Date',
    yaxis_title='Temperature [K]')

fig.show()

#### Validation set
An important step is to check identified parameters on validation set. Let's try on an new period using the last identified values.

In [ ]:
validation_set = reference_df.loc["2024-09-08 00:00":"2024-09-14 00:00"]
validation_set.loc[:,"time_sec"] = datetime_to_seconds(validation_set.index)

validation_set = validation_set.resample('5min').mean()
second_index = datetime_to_seconds(validation_set.index)

new_simulation_options_PYTH={
    "dataframe":validation_set,
    "startTime": second_index[0],
    "endTime": second_index[-1],  
}

In [ ]:
validation_results = simu_PYTH.simulate(
    parameter_dict2,
    new_simulation_options_PYTH
)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=validation_results.index,
    y=validation_results["T_ins"] ,
    fill=None,
    mode='lines',
    line_color='orange',
    name="T_insulation - Validation result"
))

fig.add_trace(go.Scatter(
    x=validation_results.index,
    y=validation_set["T_ins"],
    fill=None,
    mode='lines',
    line_color='green',
    name="T_insulation - Measurement"
))

fig.update_layout(
    title='Simulation vs. Measurement ',
    xaxis_title='Date',
    yaxis_title='Temperature [K]')

fig.show()

In [ ]:
cv_rmse(
    validation_results["T_ins"],
    validation_set["T_ins"]
)

## 4.3.3 Mixed parameters

In some calibrations, some decision variables are discrete rather than purely continuous—e.g., on/off features (booleans), choices among a few valid modes, or integers for counts. Standard continuous optimizers used earlier assume a smooth search space and therefore won’t handle these variables correctly.

To reflect realistic design decisions and configuration toggles, we now introduce a model with a boolean switch (e.g., enabling/disabling a physical effect) and restrict alpha to a small set of admissible values. This mixed-variable setup better captures practical constraints and uncertainty (e.g., unknown presence of a phenomenon, or controller setpoint options), and requires a mixed-variable optimization strategy.

In [ ]:
import numpy as np
import pandas as pd
from corrai.base.model import Model

class OpaqueWallBool(Model):
    def simulate(
        self,
        property_dict: dict,
        simulation_options: dict,
        simulation_kwargs: dict | None = None,
        **kwargs,
    ) -> pd.DataFrame:

        default_parameters = {
            "R_ext": 0.005,
            "R_int": 0.01,
            "R_concrete": 0.10,
            "R_ins": 0.32,
            "C_concrete": 2.95e6,
            "C_ins": 3.64e4,
            "alpha": 0.2,
            "S_wall": 7,
            "epsilon": 0.4,
            "fview": 0.5,
            "has_LW_radiation": True
        }
        parameters = {**default_parameters, **property_dict}

        R_ext       = parameters["R_ext"]
        R_int       = parameters["R_int"]
        R_concrete  = parameters["R_concrete"]
        R_ins       = parameters["R_ins"]
        C_concrete  = parameters["C_concrete"]
        C_ins       = parameters["C_ins"]
        alpha       = parameters["alpha"]
        S_wall      = parameters["S_wall"]
        epsilon     = parameters["epsilon"]
        fview       = parameters["fview"]
        has_LW_radiation = parameters["has_LW_radiation"]

        sigma = 5.67e-8  # W/m^2/K^4

        df = simulation_options["dataframe"]
        time  = df["time_sec"].values
        T_ext = df["T_ext"].values
        T_int = df["T_int"].values
        Q_rad = df["Pyr"].values

        startTime = simulation_options.get("startTime", time[0])
        stopTime  = simulation_options.get("stopTime",  time[-1])

        mask  = (time >= startTime) & (time <= stopTime)
        time  = time[mask]
        T_ext = T_ext[mask]
        T_int = T_int[mask]
        Q_rad = Q_rad[mask]

        # init
        T_se        = np.zeros(len(time))
        T_concrete  = np.zeros(len(time))
        T_ins       = np.zeros(len(time))
        T_interface = np.zeros(len(time))
        T_si        = np.zeros(len(time))
        T_sky       = np.zeros(len(time))

        T_se[0]        = T_ext[0]
        T_concrete[0]  = 299
        T_ins[0]       = T_int[0]
        T_interface[0] = (T_ins[0] + T_concrete[0]) / 2
        T_si[0]        = T_int[0]
        T_sky[0]       = T_int[0]

        for t in range(1, len(time)):
            dt = time[t] - time[t - 1]


            ## boolean to turn off long wave radiation exchange with environnement and sky
            T_sky[t] = 0.0552 * (T_ext[t] ** 1.5) if has_LW_radiation else 0.0

            Q_rad_sky = (
                epsilon * fview * sigma * (T_se[t - 1] ** 4 - T_sky[t] ** 4) * S_wall
                if has_LW_radiation else 0.0
            )
            Q_rad_amb = (
                epsilon * fview * sigma * (T_se[t - 1] ** 4 - T_ext[t - 1] ** 4) * S_wall
                if has_LW_radiation else 0.0
            )
            Q_rad_dir = Q_rad[t - 1] * alpha * S_wall

            T_se[t] = (
                T_ext[t - 1] / R_ext
                + T_ins[t - 1] / (R_ins / 2)
                + Q_rad_dir - Q_rad_sky - Q_rad_amb
            ) / (1 / R_ext + 1 / (R_ins / 2))

            T_interface[t] = (
                T_ins[t - 1] / (R_ins / 2) + T_concrete[t - 1] / (R_concrete / 2)
            ) / (1 / (R_concrete / 2) + 1 / (R_ins / 2))

            T_si[t] = (
                T_int[t - 1] / R_int + T_concrete[t - 1] / (R_concrete / 2)
            ) / (1 / R_int + 1 / (R_concrete / 2))

            T_ins[t] = T_ins[t - 1] + dt / C_ins * (
                (T_se[t] - T_ins[t - 1]) / (R_ins / 2)
                + (T_interface[t] - T_ins[t - 1]) / (R_ins / 2)
            )

            T_concrete[t] = T_concrete[t - 1] + dt / C_concrete * (
                (T_interface[t] - T_concrete[t - 1]) / (R_concrete / 2)
                + (T_si[t] - T_concrete[t - 1]) / (R_concrete / 2)
            )

        # output
        df_out = pd.DataFrame(
            {
                "T_concrete":  T_concrete,
                "T_interface": T_interface,
                "T_ins":       T_ins,
            },
            index=df.index[mask],
        )
        self.simulation_options = simulation_options
        return df_out

In [ ]:
mixed_params = [
    Parameter(name="alpha", values=(0.2, 0.4, 0.5), ptype="Choice", model_property="alpha"),
    Parameter(name="epsilon", interval=(0, 1), ptype="Real", model_property="epsilon"),
    Parameter(name="has_LW_radiation", ptype="Binary", model_property="has_LW_radiation"),
]

In [ ]:
from corrai.multi_optimize import Problem

obj_func = ObjectiveFunction(
    model=OpaqueWallBool(),
    simulation_options=simulation_options_PYTH,
    parameters=mixed_params,
    indicators_config={
        "T_ins": (cv_rmse, feat_train["T_ins"]),
    },
    scipy_obj_indicator="T_ins",
)

problem = Problem(
    parameters=mixed_params,
    evaluators=[obj_func],
    objective_ids=["T_ins"],
)

In [ ]:
from pymoo.termination import get_termination
from pymoo.algorithms.moo.nsga2 import RankAndCrowdingSurvival
from pymoo.core.mixed import MixedVariableGA
from pymoo.optimize import minimize

algorithm = MixedVariableGA(pop_size=50, survival=RankAndCrowdingSurvival())
termination = get_termination("n_gen", 5)

res = minimize(problem,
               algorithm,
               termination,
               seed=42,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))